In [1]:
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
import matplotlib.pyplot as plt

In [2]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

In [3]:
from treeinterpreter import treeinterpreter as ti

In [4]:
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False,index_col='building_id')
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})
df_test_values_original = pd.read_csv('test_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})

In [5]:
train_values_subset = pd.get_dummies(df_train_values_original)
geo1 = pd.get_dummies(train_values_subset["geo_level_1_id"])
train_values_subset = train_values_subset.drop(["geo_level_2_id", "geo_level_3_id", "geo_level_1_id"], axis=1)
train_values_subset = pd.concat([train_values_subset, geo1], axis=1)
train_values_subset.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,21,22,23,24,25,26,27,28,29,30
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,True,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
28830,2,10,8,7,False,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
94947,2,10,5,5,False,True,False,False,False,False,...,1,0,0,0,0,0,0,0,0,0
590882,2,10,6,5,False,True,False,False,False,False,...,0,1,0,0,0,0,0,0,0,0
201944,3,30,8,9,True,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_labels_subset = df_train_labels_original['damage_grade']

In [7]:
train_values, validation_values = (train_values_subset.iloc[0:173733], train_values_subset.iloc[173733:-1])
train_labels, validation_labels = (train_labels_subset.iloc[0:173733], train_labels_subset.iloc[173733:-1])

In [8]:
pipe = make_pipeline(StandardScaler(), 
                     RandomForestClassifier())
pipe

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [9]:
# 'randomforestclassifier__n_estimators': [150, 200, 250],
param_grid = {'randomforestclassifier__min_samples_leaf': [2, 3, 4]}
gs = GridSearchCV(pipe, param_grid)

In [ ]:
gs.fit(train_values, train_labels.values.ravel())

In [ ]:
gs.best_params_

{'randomforestclassifier__min_samples_leaf': 2}

In [38]:
in_sample_preds = gs.predict_proba(train_values)
# f1_score(train_labels, in_sample_preds, average='micro')

0.7690881985575567

In [ ]:
predictions = gs.predict(validation_values)

In [28]:
f1_score(validation_labels, predictions, average='micro')

0.6972728423912418

In [15]:
# submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [16]:
# my_submission = pd.DataFrame(data=predictions,
#                              columns=submission_format.columns,
#                              index=submission_format.index)

In [17]:
# my_submission.head()

In [18]:
# my_submission.to_csv('submission3.csv')

In [19]:
# first iteration: 
#     - sigo tutorial y elijo parametros que parecen importantes segun el informe        realizaco para la primer parte
#     - selected_features = ['geo_level_1_id', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'area_percentage', 'height_percentage', 'legal_ownership_status']
#     - param_grid = {'randomforestclassifier__n_estimators': [50, 200],
#               'randomforestclassifier__min_samples_leaf': [1, 10]}
#     - f1Score = 0.6830211702948186

In [20]:
# second iteration: 
#     - cambio geolevel 1 por 3, mas especifico
#     - selected_features = ['geo_level_3_id', 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'area_percentage', 'height_percentage', 'legal_ownership_status']
#     - param_grid = {'randomforestclassifier__n_estimators': [50, 200],
#               'randomforestclassifier__min_samples_leaf': [1, 10]}
#     - f1Score = 0.8886305117785427

In [21]:
# third iteration:
#     - igual que la segunda, solo que dropeo legal_ownership_status_v y sumo los pisos 1 2 3 y 4 como columnas 1/0
#     - f1Score = 0.8920533689433273

In [22]:
# Realizo la primer entrega, conseguimos un score del 0.6520

In [23]:
# fourth iteration:
#     - agregue todas las columnas, pasando los categorys con get_dummys. Nos da un total de 68 columnas.
#     - f1Score = 0.984297834620742
#     - puede ser un caso de overfitting?

In [127]:
# Realizo segunda entrega, conseguimos un score del 0.71
# Tener todos los features basicos mejora el score, no por mucho igualmente
# Teniendo en cuenta una diferencia de 3 veces mas features, solo hubo un aumento del 0.06 en el F1 score

SyntaxError: invalid syntax (<ipython-input-127-fec072e3d6fd>, line 1)